In [ ]:
import numpy as np
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01 
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))
        self.hidden_states = [] 
        for x in inputs:
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            self.hidden_states.append(h)
        y = np.dot(self.Why, h) + self.by
        return y

     def backward(self, inputs, targets, learning_rate):
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        dhnext = np.zeros_like(self.hidden_states[0])
        for t in reversed(range(len(inputs))):
            dy = np.copy(self.outputs[t])
            dy[np.argmax(targets[t])] -= 1 
            dWhy += np.dot(dy, self.hidden_states[t].T)
            dby += dy
            dh = np.dot(self.Why.T, dy) + dhnext
            dhraw = (1 - self.hidden_states[t] * self.hidden_states[t]) * dh 
            dbh += dhraw
            dWxh += np.dot(dhraw, inputs[t].T)
            dWhh += np.dot(dhraw, self.hidden_states[t-1].T)
            dhnext = np.dot(self.Whh.T, dhraw)
        # Clip gradients to prevent exploding gradients
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam)
        # Update weights and biases
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby